### 函数与装饰器
#### 装饰器基础知识
装饰器是可调用对象，其参数是另一个函数。严格的说，装饰器只是语法糖，它可以对被装饰的函数进行操作，比如进行替换等。

In [1]:
def decorate(func):
    print("decorate func")
    return func
@decorate
def target():
    print("running target")
target()

decorate func
running target


In [ ]:
# 上面的效果与下述写法一致
def target():
    print("running target")
target = decorate(target)
target()

#### python何时执行装饰器
装饰器的一个关键特性是，它们在被装饰的函数定义以后立即运行，这通常是在导入时发生

In [ ]:
def decorate(func):
    print("decorate func") #导入时直接输出
    return func
@decorate
def target():
    print("running target") #运行时输出
target()

#### 利用装饰器改进策略模式
上一节我们在获取最佳策略时，使用的是global()方法，并根据字符名去判断，这种方式可能会因为名称的疏忽而造成策略添加的疏漏，装饰器可以对这个地方进行改进

In [ ]:
promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func
@promotion
def vip_promo():
    print("vip")
@promotion
def old_promo():
    print("old")
@promotion
def new_promo():
    print("new")
promos

#### 变量作用域规则
python变量作用域与学过的ECMA6的作用域链很类似，变量的查找顺序：局部作用域->闭包作用域->全局作用域->内置空间作用域，逐层向上查找，直到找到为止，如果未找到，则抛出NameError<br>
 - 内置空间作用域包含了内置的关键字和函数等，如filter等。

 - 闭包作用域就是由闭包函数形成的作用域，这个概念和js中类似

In [ ]:
import builtins
print(dir(builtins))

#### 闭包
闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体只能够定义的非全局变量。

In [ ]:
#实例
def make_averager():
    series = []  # 闭包作用域的自由变量
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager
avg = make_averager()
for i in range(10):
    print(avg(i))

In [ ]:
avg.__code__.co_varnames #

In [ ]:
avg.__code__.co_freevars #自由变量

In [ ]:
avg.__closure__[0].cell_contents #闭包是一种函数，它会保留定义函数时存在的自由变量的绑定

#### nonlocal声明
当在闭包函数中对闭包作用域的不可变类型变量进行了更新，则会将其变为局部变量，隐式地创建了局部变量，就丢失了其闭包函数的性质，如下：

In [ ]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        total += new_value
        count += 1
        return total/count
    return averager
avg = make_averager()
for i in range(10):
    print(avg(i))

In [ ]:
avg.__code__.co_varnames,avg.__code__.co_freevars # 自由变量被覆盖成为局部变量了

python3引入了nonlocal声明，它的作用是把变量标记为自由变量，即使在函数中为变量赋予了新值，也会变成自由变量

In [ ]:
def make_averager(k):
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count,total,k
        total += new_value
        count += 1
        return total/count
    return averager
avg = make_averager(1)
for i in range(10):
    print(avg(i))

In [ ]:
##### late binding